In [41]:
import pandas as pd
import warnings


In [52]:
def get_metadata():
    master_table = '/mnt/c/Users/ochapman/Documents/Mesirov/pedpancan_ecdna/data/metadata/histologies.tsv'
    df = pd.read_csv(master_table,sep='\t',dtype='str')
    return df
def get_CBTN_X01_metadata():
    sample_table = '/mnt/c/Users/ochapman/Documents/Mesirov/pedpancan_ecdna/data/CBTN-AA-data/CBTN-X01-metadata.xlsx'
    df = pd.read_excel(sample_table,index_col='sample_id')
    return df
def get_CBTN_2020_metadata():
    sample_table = '/mnt/c/Users/ochapman/Documents/Mesirov/pedpancan_ecdna/data/PedPanCancer_CBTN_MasterAnalysis.xlsx'
    df = pd.read_excel(sample_table,sheet_name='1b Samples',index_col='Sample ID')
    return df
def preprocess_metadata(meta):
    meta=meta.sort_values(['sample_id','experimental_strategy']) #array(['RNA-Seq', 'WGS', 'Methylation', 'WXS', 'Targeted Sequencing'],
    meta=meta[meta.sample_type == 'Tumor']
    meta=meta[meta.Kids_First_Participant_ID.map(lambda x:x.startswith('PT_'))]
    Kids_First_Biospecimen_IDs=meta[['Kids_First_Biospecimen_ID','sample_id']].groupby('sample_id').agg({'Kids_First_Biospecimen_ID':';'.join})
    meta=meta.drop_duplicates('sample_id',keep='first')
    meta.set_index('sample_id',inplace=True)
    meta.pop('Kids_First_Biospecimen_ID')
    meta=meta.merge(Kids_First_Biospecimen_IDs,left_index=True,right_index=True)
    meta.pop('experimental_strategy')
    return meta
def get_CBTN_metadata():
    df = get_metadata()
    df = preprocess_metadata(df)
    samples = set(get_CBTN_2020_metadata().index.unique()) | set(get_CBTN_X01_metadata().index.unique())
    missing = samples - set(df.index)
    if len(missing) > 0:
        warnings.warn(f"Missing samples: {missing}")
    df=df[df.index.isin(samples)]
    return df

In [45]:
old_tbl = get_CBTN_2020_metadata()
old_tbl.head()

,Specimen_ID,Patient_ID,Cancer_Type,ecDNA,SAMPLE_TYPE,TUMOR_TYPE,AGE,cancer_predispositions,CANCER_TYPE,CANCER_TYPE_DETAILED,CNS_region,ETHNICITY,extent_of_tumor_resection,EXTERNAL_PATIENT_ID,GENDER,germline_sex_estimate,HISTOLOGY,Mutation Count,OS_MONTHS,OS_STATUS,pathology_free_text_diagnosis,PFS_MONTHS,RACE,Number of Samples Per Patient,tumor_fraction,tumor_ploidy,TUMOR_TISSUE_SITE,Replacement Sample_ID,Notes
Sample ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7316-2577,BS_AQMKA8NC,PT_00G007DM,ETMR,0,Solid Tissue,recurrence,1.0,None documented,Embryonal tumor,Embryonal Tumor with Multilayered Rosettes (ET...,Posterior fossa,Not Hispanic or Latino,Gross/Near total resection,C27429,Male,Male,Embryonal tumor with multilayer rosettes,2.0,150.0,0:LIVING,ependymoblastoma,6.0,White,2,0.613054,4.0,Cerebellum/Posterior Fossa,NaN,NaN
7316-272,BS_K07KNTFY,PT_00G007DM,ETMR,0,Solid Tissue,primary,1.0,None documented,Embryonal tumor,"Embryonal tumor with multilayer rosettes, C19M...",Posterior fossa,Not Hispanic or Latino,Gross/Near total resection,C27429,Male,Male,Embryonal tumor with multilayer rosettes,4.0,150.0,0:LIVING,embryonal tumor with abundant neuropil and tru...,6.0,White,2,0.328185,4.0,Cerebellum/Posterior Fossa,NaN,NaN
7316-447,BS_TE8QFF7T,PT_01MZ62KG,ETMR,1,Solid Tissue,progression,1.0,None documented,Embryonal tumor,"Embryonal tumor with multilayer rosettes, C19M...",Posterior fossa,Not Hispanic or Latino,Partial resection,C92004,Male,Male,Embryonal tumor with multilayer rosettes,11.0,9.0,1:DECEASED,pineoblastoma,4.0,White,1,NaN,NaN,Cerebellum/Posterior Fossa,NaN,NaN
7316-2989,BS_2VB8649B,PT_02J5CWN5,LGG,0,Solid Tissue,primary,6.0,None documented,Low-grade astrocytic tumor,"Low-grade glioma/astrocytoma, FGFR",Suprasellar,Not Hispanic or Latino,Partial resection,C90159,Male,Male,Low-grade glioma astrocytoma,7.0,96.0,1:DECEASED,low-grade glioma,49.0,Black or African American,2,1,2.0,Suprasellar/Hypothalamic/Pituitary,NaN,NaN
7316-898,BS_5TT6TT4K,PT_02J5CWN5,LGG,0,Solid Tissue,progression,6.0,None documented,Low-grade astrocytic tumor,"Low-grade glioma/astrocytoma, wildtype",Suprasellar,Not Hispanic or Latino,Partial resection,C90159,Male,Male,Low-grade glioma astrocytoma,1.0,96.0,1:DECEASED,low-grade glioma,49.0,Black or African American,2,NaN,NaN,Suprasellar/Hypothalamic/Pituitary,NaN,NaN


In [16]:
new_tbl = get_CBTN_X01_metadata()
new_tbl.head()

,gender,race,ethnicity,Kids First Participant ID,disease_type,Tumor Descriptor,sample_type,platform,Kids First Biospecimen ID,primary_site,...,Kids First Biospecimen ID Normal,library_id,aliquot_id,Kids First Biospecimen ID Tumor,reference_genome,case_id,experimental_strategy,id,name,Kids First Family ID
sample_id,,,,,,,,,,,,,,,,,,,,,
7316-7755,Male,White,Hispanic or Latino,PT_XFH4Q7SC,Glial-neuronal tumor NOS,Initial CNS Tumor,Tumor,Illumina,BS_F6R8R1KN,Temporal Lobe,...,BS_CR0XGJ55,Not_Reported,1232789,BS_F6R8R1KN,HG38,C3638832,WGS,6453ce0e0b32890db2edff05,0091d5f0-5a7b-48ec-b839-4d47d7e9b27b_summary.txt,NaN
7316-4549,Female,Other,Not Reported,PT_XM0CXQVW,Other,Initial CNS Tumor,Tumor,Illumina,BS_TWQD804Z,Other locations NOS,...,BS_7XG04FQT,Not_Reported,1228753,BS_TWQD804Z,HG38,C2603418,WGS,6453ce0e0b32890db2edfe6c,00bbd056-908d-4529-94a6-e450c24869f4_summary.txt,NaN
7316-8937,Female,White,Not Hispanic or Latino,PT_2RBR82SZ,Schwannoma,Initial CNS Tumor,Tumor,Illumina,BS_WE1KJMMQ,Spinal Cord- Cervical,...,BS_T6A7Y544,Not_Reported,1273237,BS_WE1KJMMQ,NaN,C4085076,WGS,6453ce0e0b32890db2edfe66,00bccddb-4b84-4cdd-8062-5fac4784cf4e_summary.txt,FM_AKYVF771
7316-6972,Male,Asian,Not Hispanic or Latino,PT_K84J9401,Ependymoma,Progressive,Tumor,Illumina,BS_JVDGE9S5,Cerebellum/Posterior Fossa,...,BS_6S8PDDD6,Not_Reported,1182448,BS_JVDGE9S5,HG38,C2803539,WGS,6453ce0e0b32890db2edff34,00d90281-50bd-4185-893e-aee5820b55a5_summary.txt,NaN
7316-7622,Female,White,Not Hispanic or Latino,PT_19GCSK2S,High-grade glioma/astrocytoma (WHO grade III/IV),Progressive,Tumor,Illumina,BS_AH252MTC,Cerebellum/Posterior Fossa;Hippocampus;Other l...,...,BS_22N5N72T,Not_Reported,1228644,BS_AH252MTC,HG38,C2751264,WGS,6453ce0e0b32890db2edfee1,00fff07c-ee2b-4509-8b0c-b95c7d3bb8e6_amplicon1...,NaN


In [53]:
unified_table = get_CBTN_metadata().sort_values("Kids_First_Participant_ID")
print(unified_table.shape)
unified_table.head()

(1976, 53)


/tmp/ipykernel_784/2561616889.py:30: UserWarning: Missing samples: {'7316-1769-CL-susp', '7316-3058-CL-adh', '7316-85-CL-susp', '7316-2176-CL-adh', '7316-195-CL-susp', '7316-4447-CL', '7316-4448-CL', '7316-1763-CL-adh', '7316-3235-CL', '7316-1746-CL-adh', '7316-3240-CL', '7316-1763-CL-susp', '7316-913-CL-adh', '7316-3058-CL-susp', '7316-913-CL-susp', '7316-1746-CL-susp', '7316-2189-CL-adh', '7316-3214', '7316-195-CL-adh', '7316-2151-CL-adh', '7316-4446-CL', '7316-85-CL-adh'}
  warnings.warn(f"Missing samples: {missing}")


,aliquot_id,Kids_First_Participant_ID,sample_type,composition,tumor_descriptor,primary_site,reported_gender,race,ethnicity,age_at_diagnosis_days,pathology_diagnosis,RNA_library,OS_days,OS_status,EFS_days,cohort,age_last_update_days,seq_center,cancer_predispositions,pathology_free_text_diagnosis,cohort_participant_id,extent_of_tumor_resection,CNS_region,gtex_group,gtex_subgroup,germline_sex_estimate,normal_fraction,tumor_fraction,tumor_ploidy,age_at_event_days,clinical_status_at_event,age_at_chemo_start,age_at_radiation_start,cell_line_composition,cell_line_passage,tumor_fraction_RFpurify_ABSOLUTE,tumor_fraction_RFpurify_ESTIMATE,tumor_fraction_LUMP,dkfz_v11_methylation_subclass,dkfz_v11_methylation_subclass_score,dkfz_v12_methylation_subclass,dkfz_v12_methylation_subclass_score,dkfz_v12_methylation_mgmt_status,dkfz_v12_methylation_mgmt_estimated,molecular_subtype,integrated_diagnosis,Notes,harmonized_diagnosis,molecular_subtype_methyl,broad_histology,short_histology,cancer_group,Kids_First_Biospecimen_ID
sample_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7316-2577,601591,PT_00G007DM,Tumor,Solid Tissue,Recurrence,Cerebellum/Posterior Fossa,Male,White,Not Hispanic or Latino,464,Embryonal Tumor with Multilayered Rosettes (ETMR),stranded,4583,LIVING,202,PBTA,5047,NantOmics,None documented,ependymoblastoma,C27429,Gross/Near total resection,Posterior fossa,NaN,NaN,Male,NaN,NaN,NaN,3178,Alive,482,686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"CNS Embryonal, NOS","CNS Embryonal tumor, NEC/NOS",Updated via OpenPedCan subtyping,"CNS Embryonal tumor, NEC/NOS",NaN,Embryonal tumor,Embryonal tumor,CNS Embryonal tumor,BS_FEPRNEXX;BS_AQMKA8NC
7316-272,588287,PT_00G007DM,Tumor,Solid Tissue,Initial CNS Tumor,Cerebellum/Posterior Fossa,Male,White,Not Hispanic or Latino,464,Embryonal Tumor with Multilayered Rosettes (ETMR),stranded,4583,LIVING,202,PBTA,5047,NantOmics,None documented,embryonal tumor with abundant neuropil and tru...,C27429,Gross/Near total resection,Posterior fossa,NaN,NaN,Male,NaN,NaN,NaN,464,Alive,482,686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ETMR, C19MC-altered","Embryonal tumor with multilayer rosettes, C19M...",Updated via OpenPBTA subtyping,"Embryonal tumor with multilayer rosettes, C19M...",NaN,Embryonal tumor,ETMR,Embryonal tumor with multilayer rosettes,BS_QWNBZ9RJ;BS_K07KNTFY
7316-447,1114596,PT_01MZ62KG,Tumor,Solid Tissue,Progressive,Cerebellum/Posterior Fossa,Male,White,Not Hispanic or Latino,405,Pineoblastoma,NaN,303,DECEASED,141,PBTA,708,NaN,None documented,pineoblastoma,C92004,Partial resection,Posterior fossa,NaN,NaN,Male,NaN,NaN,NaN,546,Alive,NaN,587,NaN,NaN,0.660861999999999,0.913750818333334,0.745930799398569,ETMR,1,ETMR_C19MC,0.999997356948114,unmethylated,0.0215009372949285,NaN,NaN,NaN,Pineoblastoma,NaN,Tumor of pineal region,Pineoblastoma,Pineoblastoma,BS_5AEBR8CK;BS_P39SQPTS;BS_TE8QFF7T
7316-6365,1229362,PT_01SH4F1X,Tumor,Solid Tissue,Initial CNS Tumor,Temporal Lobe,Male,White,Not Hispanic or Latino,3838,Ganglioglioma,poly-A stranded,1379,LIVING,245,PBTA,5217,NaN,None documented,"ganglioglioma with braf v600e mutation, who gr...",C2578326,Gross/Near total resection,Hemispheric,NaN,NaN,Male,NaN,NaN,NaN,3838,Alive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"GNG, wildtype","Ganglioglioma, wildtype",Updated via OpenPedCan subtyping,"Ganglioglioma, wildtype",NaN,Low-grade astrocytic tumor,Ganglioglioma,Ganglioglioma,BS_4Y2JAQTS;BS_PJ95BE63;BS_92T7PSY6
7316-2989,1116213,PT_02J5CWN5,Tumor,Solid Tissue,Initial CNS Tumor,Suprasellar/Hypothalamic/Pituitary,Male,Black or African American,Not Hispanic or Latino,2223,Low-grade glioma/astrocytoma (WHO grade I/II),NaN,2947,DECEASED,1499,PBTA,5170,NaN,None documented,low-grade glioma,C90159,Partial resection,Suprasellar,NaN,NaN,Male,NaN,NaN,NaN,2223,Alive,NaN,NaN,NaN,NaN,0.505124,0.82389717,0.63097428781851606,"LGG, PA MID",0.899,PA_INF,0.640518031141525,unmethylated,0.00740994809688996,"LGG, FGFR","Low-grade glioma, FGFR",Updated via OpenPedCa

In [40]:
outfile = '/mnt/c/Users/ochapman/Documents/Mesirov/pedpancan_ecdna/data/metadata/histologies-wgs-cohort.xlsx'
unified_table.to_excel(outfile)

In [37]:
master = get_metadata()
master[master.Kids_First_Participant_ID == 'PT_01MZ62KG']

,Kids_First_Biospecimen_ID,sample_id,aliquot_id,Kids_First_Participant_ID,experimental_strategy,sample_type,composition,tumor_descriptor,primary_site,reported_gender,race,ethnicity,age_at_diagnosis_days,pathology_diagnosis,RNA_library,OS_days,OS_status,EFS_days,cohort,age_last_update_days,seq_center,cancer_predispositions,pathology_free_text_diagnosis,cohort_participant_id,extent_of_tumor_resection,CNS_region,gtex_group,gtex_subgroup,germline_sex_estimate,normal_fraction,tumor_fraction,tumor_ploidy,age_at_event_days,clinical_status_at_event,age_at_chemo_start,age_at_radiation_start,cell_line_composition,cell_line_passage,tumor_fraction_RFpurify_ABSOLUTE,tumor_fraction_RFpurify_ESTIMATE,tumor_fraction_LUMP,dkfz_v11_methylation_subclass,dkfz_v11_methylation_subclass_score,dkfz_v12_methylation_subclass,dkfz_v12_methylation_subclass_score,dkfz_v12_methylation_mgmt_status,dkfz_v12_methylation_mgmt_estimated,molecular_subtype,integrated_diagnosis,Notes,harmonized_diagnosis,molecular_subtype_methyl,broad_histology,short_histology,cancer_group
1760,BS_5AEBR8CK,7316-447,1114596,PT_01MZ62KG,Methylation,Tumor,Solid Tissue,Progressive,Cerebellum/Posterior Fossa,Male,White,Not Hispanic or Latino,405,Pineoblastoma,NaN,303,DECEASED,141,PBTA,708,NaN,None documented,pineoblastoma,C92004,Partial resection,Posterior fossa,NaN,NaN,Male,NaN,NaN,NaN,546,Alive,NaN,587,NaN,NaN,0.660861999999999,0.913750818333334,0.745930799398569,ETMR,1,ETMR_C19MC,0.999997356948114,unmethylated,0.0215009372949285,NaN,NaN,NaN,Pineoblastoma,NaN,Tumor of pineal region,Pineoblastoma,Pineoblastoma
7356,BS_P39SQPTS,7316-447,588300,PT_01MZ62KG,RNA-Seq,Tumor,Solid Tissue,Progressive,Cerebellum/Posterior Fossa,Male,White,Not Hispanic or Latino,405,Pineoblastoma,stranded,303,DECEASED,141,PBTA,708,NantOmics,None documented,pineoblastoma,C92004,Partial resection,Posterior fossa,NaN,NaN,Male,NaN,NaN,NaN,546,Alive,NaN,587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pineoblastoma,NaN,Tumor of pineal region,Pineoblastoma,Pineoblastoma
8815,BS_TE8QFF7T,7316-447,588012,PT_01MZ62KG,WGS,Tumor,Solid Tissue,Progressive,Cerebellum/Posterior Fossa,Male,White,Not Hispanic or Latino,405,Pineoblastoma,NaN,303,DECEASED,141,PBTA,708,NantOmics,None documented,pineoblastoma,C92004,Partial resection,Posterior fossa,NaN,NaN,Male,0.672514409007,0.327485590993,2,546,Alive,NaN,587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pineoblastoma,NaN,Tumor of pineal region,Pineoblastoma,Pineoblastoma
10069,BS_Y7A7J675,7316-447,550003,PT_01MZ62KG,WGS,Normal,Peripheral Whole Blood,NaN,Peripheral Whole Blood,Male,White,Not Hispanic or Latino,405,NaN,NaN,303,DECEASED,141,PBTA,708,NantOmics,None documented,NaN,C92004,NaN,NaN,NaN,NaN,Male,NaN,NaN,NaN,546,Alive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
unified_table[unified_table.harmonized_diagnosis.isna()]

,aliquot_id,Kids_First_Participant_ID,sample_type,composition,tumor_descriptor,primary_site,reported_gender,race,ethnicity,age_at_diagnosis_days,pathology_diagnosis,RNA_library,OS_days,OS_status,EFS_days,cohort,age_last_update_days,seq_center,cancer_predispositions,pathology_free_text_diagnosis,cohort_participant_id,extent_of_tumor_resection,CNS_region,gtex_group,gtex_subgroup,germline_sex_estimate,normal_fraction,tumor_fraction,tumor_ploidy,age_at_event_days,clinical_status_at_event,age_at_chemo_start,age_at_radiation_start,cell_line_composition,cell_line_passage,tumor_fraction_RFpurify_ABSOLUTE,tumor_fraction_RFpurify_ESTIMATE,tumor_fraction_LUMP,dkfz_v11_methylation_subclass,dkfz_v11_methylation_subclass_score,dkfz_v12_methylation_subclass,dkfz_v12_methylation_subclass_score,dkfz_v12_methylation_mgmt_status,dkfz_v12_methylation_mgmt_estimated,molecular_subtype,integrated_diagnosis,Notes,harmonized_diagnosis,molecular_subtype_methyl,broad_histology,short_histology,cancer_group,Kids_First_Biospecimen_ID
sample_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
